# Базовые понятия статистики
## Задание 1. Базовое изучение

In [1]:
import pandas as pd

horse_data = pd.read_csv('horse_data.csv', na_values='?', usecols=[0, 1, 4, 10, 12, 20, 22, 23]) #, usecols=list(range(8)))
horse_data.columns = ['Surgery', 'Age', 'Pulse', 'Pain',
                      'Abdominal distension', 'Abdominocentesis appearance', 'Outcome', 'Surgical lesion']
# Общая информация
print(horse_data.info())
# Основные статистические характеристики
print(horse_data.describe())
for col in horse_data.columns:
    print(col, ':', horse_data[col].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Surgery                      298 non-null    float64
 1   Age                          299 non-null    int64  
 2   Pulse                        275 non-null    float64
 3   Pain                         244 non-null    float64
 4   Abdominal distension         243 non-null    float64
 5   Abdominocentesis appearance  135 non-null    float64
 6   Outcome                      298 non-null    float64
 7   Surgical lesion              299 non-null    int64  
dtypes: float64(6), int64(2)
memory usage: 18.8 KB
None
          Surgery        Age       Pulse        Pain  Abdominal distension  \
count  298.000000  299.00000  275.000000  244.000000            243.000000   
mean     1.395973    1.64214   71.934545    2.942623              2.259259   
std      0.489881    2.177

### Анализ данных
1. Характеристика значений по столбцам:
   Surgery, Pain, Abdominal distension, Abdominocentesis appearance, Outcome, Surgical lesion - категориальные
   Age - дискретные
   Pulse - непрерывные
2. В столбцах Age и Surgical lesion пропусков нет. Несмотря на то, что столбец Age содержит дискретные значения, в данном наборе данных он может принимать всего 2 значения. Столбец Surgical lesion также может принимать только два значения. Поэтому метрики данных столбцов нас не интересуют.
   Столбец Pulse содержит неприрывные значения, поэтому нас интересуют стандартное (28.68) отклонение, среднее (71.93) и медиана (64). Стандартное отклонение достаточно большое (сопоставимо с "расстоянием" между квартилями), поэтому более показательна в данном случае медиана.
   В остальных столбцах категориальные значения, поэтому следует использовать моду как главную характеристику значений. Однако в описании набора данных указано на наличие взаимосвязи между столбцами, поэтому мода для всего набора данных не является показательной, а ориентироваться следует на моду значений, сгруппированных по тем значениям, от которых они зависят. Например, столбец Surgery в разрезе Pain, так как между данными значениями есть зависимость: чем больше значение Pain (уровень боли), тем чаще в Surgery встречается значение 1 (Решение выполнить операцию), чем меньше - тем чаще встречается значение 2 (Решение операцию не выполнять). Ниже это продемонстрировано.

In [2]:
print(horse_data.pivot_table(index=['Pain'], columns=['Surgery'], values=['Outcome'], aggfunc = ['count']))

          count    
        Outcome    
Surgery     1.0 2.0
Pain               
1.0           9  29
2.0          34  25
3.0          39  28
4.0          28  10
5.0          33   8


## Задание 2. Выбросы

In [3]:
def find_outliers(df):
    outliers_dict = {}
    for column in df.columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (1.5 * iqr)
        upper_bound = q3 + (1.5 * iqr)
        # Удаляем выбросы
        df_without_outliers = df[(df[column].between(lower_bound, upper_bound, inclusive=True)) | (df[column].isna())]
        outliers = pd.concat([df_without_outliers, df]).drop_duplicates(keep=False)
        if outliers.empty:
            continue
        outliers_dict[column] = outliers
        print(f'> Выбросов в столбце {column} всего {len(outliers)}. Уникальные значения:\n', outliers[column].value_counts())
    
    return outliers_dict

outliers = find_outliers(horse_data)

> Выбросов в столбце Age всего 24. Уникальные значения:
 9    24
Name: Age, dtype: int64
> Выбросов в столбце Pulse всего 5. Уникальные значения:
 150.0    2
184.0    1
160.0    1
164.0    1
Name: Pulse, dtype: int64


### Анализ выбросов
1. В столбце Age (в данном наборе данных) встречается всего два значения: 1 и 9, которые значительно отличаются друг от друга, но главное - также значительно отличается и их количество в наборе данных. Значение 1 встречается 275, тогда как 9 - всего 24. Поэтому значение 9 будет считаться выбросом. Этот выброс можно считать естественным, т.к. лошадь может прожить 9 лет. Отбрасывать (удалять) строки с данным значением нецелесообразно, т.к. это приведет к потери 24 строк (8% от общего количества), что негативно скажется на репрезентативности выборки.
2. Значения выбросов в столбце Pulse выше среднего более чем в 2 раза, а нормы - в 4 раза. Высокие значения данного показателя можно интерпретировать как симптом заболевания/критического состояния здоровья. С высокими значениями в данном столбце должны коррелировать значения в столбцах Pain, Abdominal distension, Abdominocentesis appearance и Outcome. Это действительно так (см. блок ниже). Наиболее наглядна корреляция со значениями столбца Outcome: практически все лошади с высокими значениями Pulse умерли. Значит, выбросы в данном случае можно считать естественными. Выбросов всего 5, и их удаление (см. блок ниже) не влияет существенным образом на общую статистику. Кроме того, в строках с выбросами мало пропусков. Значит, если строки с выбросами не удалять, то они точно не окажут негативного влияния на общую статистику и, возможно, помогут заполнить пропуски в других строках наиболее адекватными значениями.
3. **Вывод**: принято решение не удалять строки с выбросами, так как они имеют ествественный характер и не искажают общее статистическое представление о выборке.

In [4]:
# Строки с выбросами по столбцу Pulse
print(outliers['Pulse'])
# Влияние удаления строк с выбросами в столбце Pulse на общую статистику
print(horse_data.describe() - horse_data[~horse_data.index.isin(outliers['Pulse'].index)].describe())

     Surgery  Age  Pulse  Pain  Abdominal distension  \
2        1.0    9  164.0   2.0                   4.0   
40       2.0    9  150.0   NaN                   NaN   
54       1.0    9  160.0   3.0                   4.0   
254      1.0    9  184.0   4.0                   3.0   
274      1.0    9  150.0   5.0                   2.0   

     Abdominocentesis appearance  Outcome  Surgical lesion  
2                            3.0      2.0                1  
40                           NaN      1.0                1  
54                           2.0      2.0                1  
254                          NaN      2.0                1  
274                          NaN      2.0                1  
        Surgery       Age      Pulse      Pain  Abdominal distension  \
count  5.000000  5.000000   5.000000  4.000000              4.000000   
mean  -0.003344  0.125134   1.660471  0.009290              0.016581   
std   -0.000715  0.207030   2.550897 -0.000863              0.004433   
min    0.

## Задание 3. Пропуски

In [5]:
# Скопируем DataFrame, чтобы иметь возможность обращаться к данным без пропусков
original_horse_data = horse_data.copy()

In [6]:
# Количество пропусков
print((horse_data.isna().mean() * 100).round(2).sort_values())

Age                             0.00
Surgical lesion                 0.00
Surgery                         0.33
Outcome                         0.33
Pulse                           8.03
Pain                           18.39
Abdominal distension           18.73
Abdominocentesis appearance    54.85
dtype: float64


Начнем с заполнения пустых значений в столбцах, содержащих меньшее количество пропусков. В столбцах Surgery и Outcome крайне незначительное количество пропусков, поэтому их можно как удалить, чтобы не тратить лишнюю вычислительную мощность, что наиболее догично в случае большой выборки и ограниченных ресурсов.
## Surgery и Outcome

In [7]:
horse_data.dropna(subset=['Surgery', 'Outcome'], inplace=True)

## Pain
Данный столбец содержит 18% пропусков, но он имеет связь как минимум с тремя столбцами без пропусков, а также может быть использован для заполнения пропусков еще в двух столбцах (Pulse и Abdominal distension). Поэтому имеет смысл заполнить его пропуски значениями из связанных с ним столбцов без пропусков.
Столбец принимает категориальные значения, поэтому пропуски заполняем модой.

In [8]:
horse_data['Pain'] = horse_data.groupby(['Surgery', 'Outcome', 'Surgical lesion'])['Pain'].\
    apply(lambda x: x.fillna(x.mode().iloc[0]))

### Pulse
Данный столбец содержит 8% пропусков, является одним из базовых показателей общего состояния организма, а также имеет зависимость со столбцами Age и Pain. Следовательно, можно заполнить его пропуски на основе ищеющихся данных.
Столбец содержит непрерывные значения, поэтому заполняем его медианой.

In [9]:
horse_data['Pulse'].fillna(horse_data.groupby(['Age', 'Pain'])['Pulse'].transform('median'), inplace=True)

## Abdominal distension
Значения в данном столбце имеют слабую связь со столбцом Surgery (при значении 4 значение Surgery с большой вероятностью (75% судя по блокам ниже) будет = 1, но более сильной, прямой зависимости не наблюдается), а также со столбцом Pain (прямая зависимость). Однако при изначально приблизительно сопоставимом количестве пропусков в столбцах Pain и Abdominal distension, больше половины из них - совместные, что снижает вероятность точного предсказания значений по столбцу Pain.
Однако в описании данных показатель Abdominal distension отмечен как важный, а количество пропусков в нем около 19%.
Учитывая, что:
1) Данный показатель важен;
2) Достоверно заполнить пропуски мы не можем;
3) Количество пропусков составляет 19%
будет логично удалить строки с пропусками в данном столбце.

In [10]:
horse_data.groupby(['Surgery', 'Abdominal distension'])['Outcome'].count()

Surgery  Abdominal distension
1.0      1.0                     34
         2.0                     31
         3.0                     50
         4.0                     27
2.0      1.0                     41
         2.0                     34
         3.0                     15
         4.0                     10
Name: Outcome, dtype: int64

In [11]:
horse_data.groupby(['Pain', 'Abdominal distension'])['Outcome'].count()

Pain  Abdominal distension
1.0   1.0                     24
      2.0                     12
      3.0                      2
      4.0                      1
2.0   1.0                     17
      2.0                     17
      3.0                     15
      4.0                      6
3.0   1.0                     22
      2.0                     25
      3.0                     21
      4.0                      4
4.0   1.0                      6
      2.0                      7
      3.0                     12
      4.0                     10
5.0   1.0                      6
      2.0                      4
      3.0                     15
      4.0                     16
Name: Outcome, dtype: int64

In [12]:
print('Пропусков в Pain:', original_horse_data[original_horse_data['Pain'].isna()]['Outcome'].count())
print('Пропусков в Abdominal distension:', original_horse_data[original_horse_data['Abdominal distension'].isna()]['Outcome'].count())
print('Совместных пропусков:', original_horse_data[original_horse_data['Pain'].isna() & original_horse_data['Abdominal distension'].isna()]['Outcome'].count())

Пропусков в Pain: 55
Пропусков в Abdominal distension: 56
Совместных пропусков: 34


In [13]:
# Решение по столбцу
horse_data.dropna(subset=['Abdominal distension'], inplace=True)

## Abdominocentesis appearance
Из сказанного об этом столбце в описании данных мы не можем сделать выводов о его связи с другими столбцами. Кроме того, в данном столбце 55% значений отсутствует. Следовательно, в рамках нашей выборки этот столбец нельзя считать значимым и в целях избавления от лишних данных его можно удалить. Учитывая, что это последний столбец с пропусками, его можно удалить командой, приведенной ниже: 

In [14]:
horse_data.dropna(axis='columns', inplace=True)
horse_data.count()

Surgery                 242
Age                     242
Pulse                   242
Pain                    242
Abdominal distension    242
Outcome                 242
Surgical lesion         242
dtype: int64